## Bayesian Optimization vs. Grid Search

This notebook is a sanity check to ensure that Bayesian optimization is finding some approximation of the global maxima. 

Data required by the notebook:
1. An optima determined by Bayesian optimization (rtms_optimize notebook for details)
2. A set of data points corresponding to input/output pairs (sample_stim_objective_func for details)

We will first visualize the objective function generated by sample_stim_objective_func, then check how closely does Bayesian optimization align with the grid-based observation.

In [7]:
import os
import numpy as np
from fieldopt import geolib

In [8]:
#Results generated from rtms_optimize notebook
best_pt = -8192.155459912105
best_coord = np.array([30.10740575, -10.48380714, 106.87593243])

First we re-generate the sampling set used by sample_stim_objective_func, then load in the outputs of that data according to the order specified there:

In [16]:
proj_dir =  '/projects/jjeyachandra/rtms_optimize'
mesh_file = os.path.join(proj_dir,'data','simnibs_output','sub-CMH090.msh')
coil_file = os.path.join(proj_dir,'resources','coils','Magstim_70mm_Fig8.nii.gz')
tet_file = os.path.join(proj_dir,'output','tetra_parcels')
C_file = os.path.join(proj_dir,'output','quadratic_vec')
iR_file = os.path.join(proj_dir,'output','inverse_rot')
b_file = os.path.join(proj_dir,'output','param_bounds')
output_dir = os.path.join(proj_dir,'objective_grid','scores')

In [11]:
C = np.fromfile(C_file)
iR = np.fromfile(iR_file).reshape(3,3)
b = np.fromfile(b_file).reshape(3,2)

In [12]:
fidelity = 30
x_samps = np.linspace(b[0,0],b[0,1],fidelity)
y_samps = np.linspace(b[1,0],b[1,1],fidelity)
rot_samps = np.linspace(0,180,fidelity)
XX,YY,RR = np.meshgrid(x_samps,y_samps,rot_samps)
XX = XX.flatten().reshape((fidelity**3,1))
YY = YY.flatten().reshape((fidelity**3,1))
RR = RR.flatten().reshape((fidelity**3,1))
sample_set = np.concatenate((XX,YY,RR),axis=1)

Here <code>sample_set</code> contains rows where each row corresponds to a point on the 3D-cube sampling space. The ordering of sample_set corresponds to the ordering of the files in the output directory

In [23]:
outfile = os.path.join(output_dir, 'fx_{}.npy')

In [24]:
f_x_list = []
for i in np.arange(0,fidelity**3):
    f_x_list.append(np.load(outfile.format(i)))

In [30]:
#To skip reloading all fidelity**3 files
combined_out='../../objective_grid/combined_scores/fid_30_sub-CMH090_scores'
f_x = np.array(f_x_list)
np.save(combined_out,f_x)

In [32]:
f_x = np.load(combined_out + '.npy')

Now <code> f_x </code> contains an entry corresponding to each entry by order in sample_set. Let's first pick the optimum then compare it to what Bayesian Optimization generates

In [35]:
grid_ind = np.argmax(f_x)
grid_coords = sample_set[grid_ind]
grid_pt = f_x[grid_ind]
print(grid_coords)
print(grid_pt)

[ 30.10740575 -10.48380714 105.51724138]
[8192.37259141]


In [37]:
#Compare with bayesian
print(best_coord)
print(best_pt)

[ 30.10740575 -10.48380714 106.87593243]
-8192.15545991


In [39]:
#Visualize objective function TODO

The final check would be to make sure that this result is reproducible upon multiple starts of Bayesian optimization. If they converge to the same answer, then we can extend this across multiple participants and objective landscapes (weight functions) to ensure full generalizability.